In [10]:
from netCDF4 import Dataset
from netCDFfunc.preprocessing import download_data
from netCDFfunc.utility import *

import scipy.ndimage as ndimage
import numpy as np

import os

In [2]:
download_path = '/Volumes/T7/download_data'
output_path = '/Volumes/T7/output_data'

In [3]:
download_data(download_path=download_path)

[20220813|AVHRR] is already downloaded!
[20220813|CMC] is already downloaded!
[20220813|DMI] is already downloaded!
[20220813|GAMSSA] is already downloaded!
[20220813|MUR25] is already downloaded!
[20220813|MUR] is already downloaded!
[20220813|MWIR] is already downloaded!
[20220813|MW] is already downloaded!
[20220813|NAVO] is already downloaded!
[20220813|OSPON] is already downloaded!
[20220813|OSPO] is already downloaded!
[20220813|OSTIA] is already downloaded!
[20220814|AVHRR] is already downloaded!
[20220814|CMC] is already downloaded!
[20220814|DMI] is already downloaded!
[20220814|GAMSSA] is already downloaded!
[20220814|MUR25] is already downloaded!
[20220814|MUR] is already downloaded!
[20220814|MWIR] is already downloaded!
[20220814|MW] is already downloaded!
[20220814|NAVO] is already downloaded!
[20220814|OSPON] is already downloaded!
[20220814|OSPO] is already downloaded!
[20220814|OSTIA] is already downloaded!
[20220815|AVHRR] is already downloaded!
[20220815|CMC] is alre

In [11]:
def get_date(file_name):
    return file_name[:8]

In [12]:
def get_meta_data(raw_data_path, date, file_name):
        
    file_name = date + file_name
    data_file = os.path.join(raw_data_path, file_name)
    ds_in = Dataset(data_file, 'r', format='NETCDF4')
    
    sst = ds_in.variables['analysed_sst'][:].data[0]
    mask = ds_in.variables['mask'][:].data[0]
    ice = ds_in.variables['sea_ice_fraction'][:].data[0]
    
    try :
        if 'MW_IR' in file_name :
            grid = 0.08789
        else :
            grid = np.float32(ds_in.geospatial_lat_resolution)
    except :
        grid = 0.05
        
    lat = ds_in.variables['lat'][:].data
    lon = ds_in.variables['lon'][:].data
    
    ds_in.close()
    
    return {'sst':sst, 'mask':mask, 'ice':ice, 'lat':lat, 'lon':lon, 'grid':grid}

In [13]:
def preprocessing_dataset(ds_name, data_dic) :
    
    sst = data_dic['sst'].copy()
    ice = data_dic['mask'].copy()

    
    if ds_name == 'CMC' :
        cmc_land = data_dic['mask'].copy()
        np.place(cmc_land, cmc_land[:,:] != 2, False)
        np.place(cmc_land, cmc_land[:,:] == 2, True)
        sst = masking(sst, cmc_land, fill_value=-32767)
        
        sst = np.roll(sst, -1, axis=0)
        ice = np.roll(ice, -1, axis=0)
        
        sst = np.roll(sst, -1, axis=1)
        ice = np.roll(ice, -1, axis=1)
        
    if ds_name == 'DMI' :
        sst = np.roll(sst, 18, axis=0)
        ice = np.roll(ice, 18, axis=0)
        
    if ds_name == 'NAVO':
        sst = np.flip(sst, axis=0)
        ice = np.flip(ice, axis=0)
        
        sst = np.roll(sst, -1, axis=0)
        ice = np.roll(ice, -1, axis=0)
        
        sst = np.roll(sst, -1, axis=1)
        ice = np.roll(ice, -1, axis=1)
    
    np.place(sst, sst[:,:] <= -32767., 500)
    sst = sst - 273.15
    
    # ice
    if ds_name == 'AVHRR' :
        ice = data_dic['ice'].copy()
        np.place(ice, ice[:,:] != -128, True)
        np.place(ice, ice[:,:] == -128, False)

    elif ds_name == 'CMC' or ds_name == 'GAMSSA':
        np.place(ice, ice[:,:] != 8, False)
        np.place(ice, ice[:,:] == 8, True)
        
    elif ds_name == 'DMI' or ds_name == 'MUR25' or ds_name == 'MUR' or ds_name == 'MW' or ds_name == 'OSTIA' or ds_name == 'MWIR':
        np.place(ice, ice[:,:] != 9, False)
        np.place(ice, ice[:,:] == 9, True)
        
    elif ds_name == 'OSPO' or ds_name == 'OSPON':
        np.place(ice, ice[:,:] != 4, False)
        np.place(ice, ice[:,:] == 4, True)
    elif ds_name == 'NAVO' :
        np.place(ice, ice[:,:], False)
        
    return sst, ice

In [16]:
#dataset_names = ['AVHRR', 'CMC', 'DMI', 'GAMSSA', 'MUR25', 'MUR', 'MWIR', 'MW', 'NAVO', 'OSPON', 'OSPO', 'OSTIA']
grid_set = ['0.01', '0.05', '0.054', '0.081', '0.08789', '0.1', '0.25']
dataset_names = ['AVHRR']

for ds_name in dataset_names :
    
    print(f'{ds_name} processing...')
        
    # anomally and grade save location
    target_path_base = os.path.join(output_path, ds_name)
    
    if not os.path.exists(target_path_base) : 
        os.mkdir(target_path_base)
        
    # get recent 6 days' raw nc file date list
    raw_data_path = os.path.join(download_path, ds_name)
    raw_data_file_name = os.listdir(raw_data_path)[-1][8:]
    
    raw_data_date_list = sorted(list(map(get_date, os.listdir(raw_data_path))))[-10:]
    
    # process data by date
    for date in raw_data_date_list:
        
        if int(date) <= 20201231 :
            period = 1
        else :
            period = 2
            
        ds_mean = dict()
        ds_ice = dict()
        ds_pctl = dict()
        
        # load base data
        for grid_size in grid_set :
            ds_mean[grid_size] = Dataset(f'/Volumes/T7/base_data/{period}/avg/{grid_size}/avg_{period}_rok_{date[4:]}_{grid_size}.nc', 'r', format='NETCDF4').variables['avgsst'][:].data[0]
            ds_ice[grid_size] = Dataset(f'/Volumes/T7/base_data/{period}/ice/{grid_size}/ice_{period}_rok_{date[4:]}_{grid_size}.nc', 'r', format='NETCDF4').variables['ice'][:].data[0]
            ds_pctl[grid_size] = Dataset(f'/Volumes/T7/base_data/{period}/pctl/{grid_size}/pctl_{period}_rok_{date[4:]}_{grid_size}.nc', 'r', format='NETCDF4').variables['pctlsst'][:].data[0]
        
        # check ouput data status
        output_date_list = sorted(list(map(get_date, os.listdir(target_path_base))))[-6:]
        target_path = os.path.join(target_path_base, date)
        if date not in output_date_list and not os.path.exists(target_path):
            os.mkdir(target_path)
            
        # overlap check
        if len(os.listdir(target_path)) == 4 : continue
        
        # data processing
        meta_data_dic = get_meta_data(raw_data_path, date, raw_data_file_name)
        
        grid = meta_data_dic['grid']
        sst, ice = preprocessing_dataset(ds_name, meta_data_dic)
        
        sst = cropping(sst, 'rok', grid)
        ice = cropping(ice, 'rok', grid).astype(bool)
        
        if ds_name == 'MWIR':
            ice = ice[:-1,:]
            sst = sst[:-1,:]
        
        mean = ds_mean[str(grid)]
        pctl = ds_pctl[str(grid)]
        base_ice = ds_ice[str(grid)].astype(bool)
        
        ice = base_ice + ice
    
        anomaly = get_anomaly_grade(sst, ice, mean, pctl)
        grade = get_anomaly_grade(sst, ice, mean, pctl, is_grade=True)
        
        to_img(anomaly, output_path=f'/Volumes/T7/other_data/anomaly_img/{ds_name}', is_anomaly=True, save_img=True)
        
        if int(date) == 20220222:
            break

AVHRR processing...
